<a href="https://colab.research.google.com/github/villafue/Capstone_2_Netflix/blob/main/Springboard/Tutorial/LinkedIn_Learning/RecSys-Materials/DeepLearningIntro/PoliticsExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Exercise

## Predict political party based on votes

As a fun little example, we'll use a public data set of how US congressmen voted on 17 different issues in the year 1984. Let's see if we can figure out their political party based on their votes alone, using a deep neural network!

For those outside the United States, our two main political parties are "Democrat" and "Republican." In modern times they represent progressive and conservative ideologies, respectively.

Politics in 1984 weren't quite as polarized as they are today, but you should still be able to get over 90% accuracy without much trouble.

Since the point of this exercise is implementing neural networks in Keras, I'll help you to load and prepare the data.

Let's start by importing the raw CSV file using Pandas, and make a DataFrame out of it with nice column labels:

In [2]:
import pandas as pd
%matplotlib inline
import numpy as np

feature_names =  ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv('https://raw.githubusercontent.com/villafue/Capstone_2_Netflix/main/Springboard/Tutorial/LinkedIn_Learning/RecSys-Materials/DeepLearningIntro/house-votes-84.data.txt', na_values=['?'], 
                          names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


We can use describe() to get a feel of how the data looks in aggregate:

In [4]:
voting_data.describe().T

,count,unique,top,freq
party,435,2,democrat,267
handicapped-infants,423,2,n,236
water-project-cost-sharing,387,2,y,195
adoption-of-the-budget-resolution,424,2,y,253
physician-fee-freeze,424,2,n,247
el-salvador-aid,420,2,y,212
religious-groups-in-schools,424,2,y,272
anti-satellite-test-ban,421,2,y,239
aid-to-nicaraguan-contras,420,2,y,242
mx-missle,413,2,y,207


In [5]:
voting_data.shape

(435, 17)

We can see there's some missing data to deal with here; some politicians abstained on some votes, or just weren't present when the vote was taken. We will just drop the rows with missing data to keep it simple, but in practice you'd want to first make sure that doing so didn't introduce any sort of bias into your analysis (if one party abstains more than another, that could be problematic for example.)

In [7]:
voting_data.dropna(inplace=True)
voting_data.describe().T

,count,unique,top,freq
party,232,2,democrat,124
handicapped-infants,232,2,n,136
water-project-cost-sharing,232,2,n,125
adoption-of-the-budget-resolution,232,2,y,123
physician-fee-freeze,232,2,n,119
el-salvador-aid,232,2,y,128
religious-groups-in-schools,232,2,y,149
anti-satellite-test-ban,232,2,y,124
aid-to-nicaraguan-contras,232,2,y,119
mx-missle,232,2,n,119


In [8]:
voting_data.shape

(232, 17)

Our neural network needs normalized numbers, not strings, to work. So let's replace all the y's and n's with 1's and 0's, and represent the parties as 1's and 0's as well.

In [9]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

In [10]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


Finally let's extract the features and labels in the form that Keras will expect:

In [11]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values

OK, so have a go at it! You'll want to refer back to the slide on using Keras with binary classification - there are only two parties, so this is a binary problem. This also saves us the hassle of representing classes with "one-hot" format like we had to do with MNIST; our output is just a single 0 or 1 value.

Also refer to the scikit_learn integration slide, and use cross_val_score to evaluate your resulting model with 10-fold cross-validation.

**If you're using tensorflow-gpu on a Windows machine** by the way, you probably *do* want to peek a little bit at my solution - if you run into memory allocation errors, there's a workaround there you can use.

Try out your code here:

In [16]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [14]:
X_train, X_test, y_train, y_test = train_test_split(all_features, all_classes, test_size=0.33, random_state=29)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels,
                    batch_size=100,
                    epochs=10,
                    verbose=2,
                    validation_data=(test_images, test_labels))

score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [19]:
def create_model():
  model = Sequential()
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(4, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  return model

estimator = KerasClassifier(build_fn=create_model, nb_epoch=100, verbose=2)

cv_scores = cross_val_score(estimator, X_train, y_train, cv=10)
# test_results = (y_test-estimator.predict(X_test))
print('\n', '-' * 136)
print('The training RMSE is: {:.4f}'.format(cv_scores))
print('The testing RMSE is: {:.4f}'.format(test_results))
print('\n', '-' * 136)



5/5 - 1s - loss: 0.6930 - accuracy: 0.5612
1/1 - 0s - loss: 0.6919 - accuracy: 0.6250
5/5 - 1s - loss: 0.6929 - accuracy: 0.4964
1/1 - 0s - loss: 0.6937 - accuracy: 0.3750
5/5 - 1s - loss: 0.6932 - accuracy: 0.4029
1/1 - 0s - loss: 0.6929 - accuracy: 0.5625
5/5 - 1s - loss: 0.6932 - accuracy: 0.4604
1/1 - 0s - loss: 0.6933 - accuracy: 0.2500
5/5 - 1s - loss: 0.6931 - accuracy: 0.4820
1/1 - 0s - loss: 0.6915 - accuracy: 0.6875
5/5 - 1s - loss: 0.6925 - accuracy: 0.6214
1/1 - 0s - loss: 0.6926 - accuracy: 0.4000
5/5 - 1s - loss: 0.6932 - accuracy: 0.4071
1/1 - 0s - loss: 0.6930 - accuracy: 0.5333
5/5 - 1s - loss: 0.6929 - accuracy: 0.6929
1/1 - 0s - loss: 0.6923 - accuracy: 0.5333
5/5 - 1s - loss: 0.6930 - accuracy: 0.5929
1/1 - 0s - loss: 0.6932 - accuracy: 0.4000
5/5 - 1s - loss: 0.6928 - accuracy: 0.5643
1/1 - 0s - loss: 0.6921 - accuracy: 0.6000

 ----------------------------------------------------------------------------------------------------------------------------------------


TypeError: ignored

## My implementation is below

# No peeking!

![title](https://github.com/villafue/Capstone_2_Netflix/blob/main/Springboard/Tutorial/LinkedIn_Learning/RecSys-Materials/DeepLearningIntro/peek.jpg?raw=1)

In [20]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()
    #16 feature inputs (votes) going into an 32-unit layer 
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    # Another hidden layer of 16 units
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

0.9438405811786652

94% without even trying too hard! Did you do better? Maybe more neurons, more layers, or Dropout layers would help even more.